In [2]:
import nltk
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/mirakotamaki/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
#General
import numpy as np
import pandas as pd
import json
import csv
import glob

#Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

#spacy
import spacy
from nltk.corpus import stopwords

#visualization
import pyLDAvis
import pyLDAvis.gensim_models

In [4]:
#converting csv file to json
with open ("/Users/mirakotamaki/Desktop/python/python/mirandata1csv.txt", "r") as f:
    reader = csv.reader(f)
    next(reader)
    data = {"columns": []}
    for row in reader:
        data["columns"].append({"column1":
        row[0], "column2": row[1]})
    
        
with open ("columns.json", "w") as f:
    json.dump(data, f, indent=4)

In [5]:
#preparations
def load_data(file):
    with open (file, "r", encoding="utf-8") as f:
        data = json.load(f)
    return(data)

def write_data(file, data):
    with open (file, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4)
        

In [6]:
stopwords = stopwords.words("english")

In [7]:
print (stopwords)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [27]:
data = load_data("columns.json")["columns"]

from pandas import json_normalize

#converting columns to string
flatten_data = json_normalize(data)

df = pd.DataFrame(flatten_data, columns = ["column1"])
df = [str(i) for i in df.values] 


res = [''.join(idx for idx in sub) for sub in df]

while("''" in df) :
    df.remove("''")


In [9]:
#lemmatization with spacy
def lemmatization(texts, allowed_postags=["NOUN", "ADJ", "VERB", "ADV"]):
    nlp = spacy.load("en_core_web_sm")
    texts_out = []
    for text in texts:
        doc = nlp(text)
        new_text = []
        for token in doc:
            if token.pos_ in allowed_postags:
                new_text.append(token.lemma_)
        final = " ".join(new_text)
        texts_out.append(final)
    return (texts_out)


lemmatized_texts = lemmatization(df)
print (lemmatized_texts)

['woman', "'", 'world', 'congress', '', 'finnish', 'democratic', 'woman', "'", 'association', 'tel', 'confederation', 'europ', 'ene', "' '", "l '", 'agriculture', 'otaniemi', 'pellervo', 'seura', '', 'eskelinen', 'tel', '', 'international', '', 'conference', '', 'finnish', '', 'association', '', '', 'tel', 'boy', 'scout', 'world', 'congress', '', 'suoman', 'partiopoikaj', 'rj', 'h', 'meent', 'tel', 'lady', 'round', 'table', 'circle', '', "'", '', 'tel', 'september', 'ix', 'finnish', 'trade', 'fair', 'exhibition', 'hall', '', 'finnish', 'fair', 'corporation', 'tel', '', 'boat', '', 'camping', 'show', 'exhibition', 'hall', '', 'finnish', 'fair', 'corporation', 'tel', "'", 'helsinki', 'festival', 'helsinki', '', "'", 'ti', 'unionink', 'tel', '', 'helsinki', 'day', 'helsingin', 'kaupunki', 'tel', '', 'international', 'youth', 'hostel', 'organization', 'conference', 'otaniemi', 'suoman', 'retkeilymajaj', 'rj', 'tel', '', '', 'september', 'helsinki', 'international', 'trade', 'fair', 'exhibi

In [14]:
#stopwords
def gen_words(texts):
    final = []
    for text in texts:
        new = gensim.utils.simple_preprocess(text, deacc=True)
        final.append(new)
    return (final)

data_words = gen_words(lemmatized_texts)
print (data_words)

[['woman'], [], ['world'], ['congress'], [], ['finnish'], ['democratic'], ['woman'], [], ['association'], ['tel'], ['confederation'], ['europ'], ['ene'], [], [], ['agriculture'], ['otaniemi'], ['pellervo'], ['seura'], [], ['eskelinen'], ['tel'], [], ['international'], [], ['conference'], [], ['finnish'], [], ['association'], [], [], ['tel'], ['boy'], ['scout'], ['world'], ['congress'], [], ['suoman'], ['partiopoikaj'], ['rj'], [], ['meent'], ['tel'], ['lady'], ['round'], ['table'], ['circle'], [], [], [], ['tel'], ['september'], ['ix'], ['finnish'], ['trade'], ['fair'], ['exhibition'], ['hall'], [], ['finnish'], ['fair'], ['corporation'], ['tel'], [], ['boat'], [], ['camping'], ['show'], ['exhibition'], ['hall'], [], ['finnish'], ['fair'], ['corporation'], ['tel'], [], ['helsinki'], ['festival'], ['helsinki'], [], [], ['ti'], ['unionink'], ['tel'], [], ['helsinki'], ['day'], ['helsingin'], ['kaupunki'], ['tel'], [], ['international'], ['youth'], ['hostel'], ['organization'], ['conferen

In [15]:
id2word = corpora.Dictionary(data_words)

corpus = []
for text in data_words:
    new = id2word.doc2bow(text)
    corpus.append(new)
    

word = id2word

In [22]:
#topic model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=8,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha="auto")

In [26]:
#visualization
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word, mds="mmds", R=10)
vis

/Users/mirakotamaki/opt/anaconda3/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(
/Users/mirakotamaki/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/mirakotamaki/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/mirakotamaki/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/mirakot

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.064066  0.373425       1        1  82.409397
6      0.066099 -0.253122       2        1   6.068315
5     -0.236789 -0.006866       3        1   3.977591
7     -0.049153  0.057271       4        1   1.944369
4      0.171715 -0.036221       5        1   1.589133
2      0.031172 -0.029163       6        1   1.573005
3      0.035720  0.010997       7        1   1.232692
0     -0.082831 -0.116322       8        1   1.205498, topic_info=              Term        Freq       Total Category  logprob  loglift
396         museum  184.000000  184.000000  Default  10.0000  10.0000
6              tel  124.000000  124.000000  Default   9.0000   9.0000
438     restaurant   82.000000   82.000000  Default   8.0000   8.0000
81            have   94.000000   94.000000  Default   7.0000   7.0000
72            city   53.000000   53.000000  Default   6.0000   6.0000
...            ...         ...         ...      ...      ...      ...
12        pellervo    0.035196    0.576784   Topic8  -8.3808   1.6217
2734     accompany    0.035197    6.430367   Topic8  -8.3808  -0.7895
2689         helka    0.035197    4.159797   Topic8  -8.3808  -0.3540
699   sinebrychoff    0.035197   15.470498   Topic8  -8.3808  -1.6675
2715       grocery    0.035196    3.619047   Topic8  -8.3808  -0.2147

[120 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
2734      1  0.777561   accompany
2734      2  0.155512   accompany
385       3  0.910722    addition
1167      1  0.997614       adult
1229      1  0.999427          am
...     ...       ...         ...
616       5  0.854660  university
110       5  0.994112        work
1         4  0.833968       world
2158      1  0.997920         www
549       6  0.615619         zoo

[61 rows x 3 columns], R=10, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 7, 6, 8, 5, 3, 4, 1])